In [36]:
import pandas as pd
import numpy as np

training_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

In [37]:
corr_matrix = training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [38]:
training_set.isna().sum()
training_set['Cabin'].value_counts()
#training_set['Embarked'].value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [39]:
########## HANDLING MISSING DATA  AND EXTRACINTG FEATURES ##########
from sklearn.impute import SimpleImputer

mean_imputer = SimpleImputer(strategy="mean")
most_frequent_imputer = SimpleImputer(strategy='most_frequent')

training_set[['Age']] = mean_imputer.fit_transform(training_set[['Age']])
training_set[['Embarked']] = most_frequent_imputer.fit_transform(training_set[['Embarked']])

training_set['Cabin'] = training_set['Cabin'].fillna('Missing')
training_set['Deck'] = training_set['Cabin'].str[0]
training_set['HasCabin'] = (training_set['Cabin'] != 'Missing').astype(int)
training_set['CabinCount'] = training_set['Cabin'].apply(lambda x:0 if x != 'Missing' else len(x.split()))
training_set['Title'] = training_set['Name'].str.extract(' ([A-Za-z]+)\\.', expand=False)
training_set = training_set.drop(columns=['Name']) #===== DROPPING THE TITLE COLUMN BECAUSE I EXTRACTED TITLE FROM IT
training_set = training_set.drop(columns=['Cabin']) #===== DROPPING THE CABIN COLUMN BECAUSE I EXTRACTED USEFUL FEATURES OUT OF IT
training_set = training_set.drop(columns=['Ticket']) #===== DROPPING THE TICKET COLUMN BECAUSE IT SERVES NO PURPOSE

In [40]:
########## ENCODING CATEGORICAL DATA ##########
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

#========== APPLYING LABEL ENCODING TO 'SEX' COLUMN =========#
le = LabelEncoder()
training_set["Sex"] = le.fit_transform(training_set["Sex"])
#========== APPLYING ONE HOT ENCODING TO 'DECK', 'TITLE' AND 'EMBARKED' COLUMNS =========#
encoder = OneHotEncoder(sparse_output = False)
encoded = encoder.fit_transform(training_set[["Deck", "Title", "Embarked"]])

# Convert back to DataFrame with column names
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(["Deck", "Title", "Embarked"]))

# Combine with original data (dropping the old column)
training_set = pd.concat([training_set.reset_index(drop=True), encoded_df], axis=1).drop(columns=["Deck", "Title", "Embarked"])

# # Define ColumnTransformer
# ct = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(drop = 'first', handle_unknown='ignore'), ['Deck'])
#     ],
#     remainder='passthrough'  # keep other columns as they are
# )
# training_set = np.array(ct.fit_transform(training_set))

In [41]:
training_set.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,HasCabin,CabinCount,...,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,1,22.0,1,0,7.2500,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,1,1,0,38.0,1,0,71.2833,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,1,3,0,26.0,0,0,7.9250,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,1,1,0,35.0,1,0,53.1000,1,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0,3,1,35.0,0,0,8.0500,0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
